#### make datasets

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

In [2]:
MODEL = "microsoft/graphcodebert-base"
TRAIN_LV1_INPUT = "./data/train_data_lv1.csv"
TRAIN_LV2_INPUT = "./data/train_data_lv2.csv"
TRAIN_LV3_INPUT = "./data/train_data_lv3.csv"
VALID_LV1_INPUT = "./data/valid_data_lv1.csv"
VALID_LV2_INPUT = "./data/valid_data_lv2.csv"
VALID_LV3_INPUT = "./data/valid_data_lv3.csv"
MAX_LEN = 512

train_dataset_lv1 = load_dataset("csv", data_files=TRAIN_LV1_INPUT)['train']
train_dataset_lv2 = load_dataset("csv", data_files=TRAIN_LV2_INPUT)['train']
train_dataset_lv3 = load_dataset("csv", data_files=TRAIN_LV3_INPUT)['train']
valid_dataset_lv1 = load_dataset("csv", data_files=VALID_LV1_INPUT)["train"]
valid_dataset_lv2 = load_dataset("csv", data_files=VALID_LV2_INPUT)["train"]
valid_dataset_lv3 = load_dataset("csv", data_files=VALID_LV3_INPUT)["train"]
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.truncation_side='left'

def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True,)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

train_dataset_lv1 = train_dataset_lv1.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
train_dataset_lv2 = train_dataset_lv2.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
train_dataset_lv3 = train_dataset_lv3.map(example_fn, remove_columns=['code1', 'code2', 'similar'])

valid_dataset_lv1 = valid_dataset_lv1.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
valid_dataset_lv2 = valid_dataset_lv2.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
valid_dataset_lv3 = valid_dataset_lv3.map(example_fn, remove_columns=['code1', 'code2', 'similar'])

Using custom data configuration default-9107b79668af0ad1
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-9107b79668af0ad1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-b804112fdc17747b
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-b804112fdc17747b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-a069df529ed2441c
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-a069df529ed2441c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


...

  0%|          | 0/90000 [00:00<?, ?ex/s]

  0%|          | 0/90000 [00:00<?, ?ex/s]

  0%|          | 0/30000 [00:00<?, ?ex/s]

  0%|          | 0/9000 [00:00<?, ?ex/s]

  0%|          | 0/9000 [00:00<?, ?ex/s]

In [3]:
train_dataset_lv1.save_to_disk('./data/train_dataset_lv1')
train_dataset_lv2.save_to_disk('./data/train_dataset_lv2')
train_dataset_lv3.save_to_disk('./data/train_dataset_lv3')
valid_dataset_lv1.save_to_disk('./data/valid_dataset_lv1')
valid_dataset_lv2.save_to_disk('./data/valid_dataset_lv2')
valid_dataset_lv3.save_to_disk('./data/valid_dataset_lv3')